In [1]:
import pandas as pd
from math import log
import operator
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"C:\Users\Ogivano\Desktop\data_wawancara.csv",  sep=';', index_col=0)

In [3]:
print(df)

      DIVISI PENDIDIKAN PRODI  IPK PEMBERKASAN WAWANCARA  AGE   STATUS  HASIL
CAPEG                                                                        
1         TL    SARJANA    AK  2.5     LENGKAP         A   25  MENIKAH  LOLOS
2        MRK     MASTER  MATH  3.2     LENGKAP         B   23   LAJANG  LOLOS
3         CS    SARJANA  MATH  3.4     LENGKAP         A   26   LAJANG  LOLOS
4         CS    SARJANA  MATH  2.7     LENGKAP         B   26   LAJANG  LOLOS
5        MRK     MASTER    MM  3.3     LENGKAP         A   29   LAJANG  LOLOS
6        MRK     MASTER    MM  3.4     LENGKAP         A   33  MENIKAH  LOLOS
7         TL   DIPLOMA3    MM  2.3     LENGKAP         C   23  MENIKAH  GAGAL
8         CC   DIPLOMA1  COMP  3.2      KURANG         A   23  MENIKAH  LOLOS
9         CC   DIPLOMA1  COMP  2.4     LENGKAP         C   29   LAJANG  GAGAL
10        CS    SARJANA    AK  3.3     LENGKAP         A   26   LAJANG  LOLOS
11        CS    SARJANA    AK  2.4      KURANG         B   26   

In [4]:
df.loc[df['HASIL'] == 'LOLOS', 'HASIL'] = 1
df.loc[df['HASIL'] == 'GAGAL', 'HASIL'] = 0
print(df)

      DIVISI PENDIDIKAN PRODI  IPK PEMBERKASAN WAWANCARA  AGE   STATUS  HASIL
CAPEG                                                                        
1         TL    SARJANA    AK  2.5     LENGKAP         A   25  MENIKAH      1
2        MRK     MASTER  MATH  3.2     LENGKAP         B   23   LAJANG      1
3         CS    SARJANA  MATH  3.4     LENGKAP         A   26   LAJANG      1
4         CS    SARJANA  MATH  2.7     LENGKAP         B   26   LAJANG      1
5        MRK     MASTER    MM  3.3     LENGKAP         A   29   LAJANG      1
6        MRK     MASTER    MM  3.4     LENGKAP         A   33  MENIKAH      1
7         TL   DIPLOMA3    MM  2.3     LENGKAP         C   23  MENIKAH      0
8         CC   DIPLOMA1  COMP  3.2      KURANG         A   23  MENIKAH      1
9         CC   DIPLOMA1  COMP  2.4     LENGKAP         C   29   LAJANG      0
10        CS    SARJANA    AK  3.3     LENGKAP         A   26   LAJANG      1
11        CS    SARJANA    AK  2.4      KURANG         B   26   

In [29]:
def cekDigit(list):
     for x in list:
        try:
            float(x)
            return True
        except:
            return False

def entropy(plus, minus):
    if (plus == 0 or minus == 0):
        return (0)
    else:
        penyebut = plus + minus
        hasil = -1 * (float(plus/penyebut) * log(float(plus/penyebut), 2) + float(minus/penyebut) * log(float(minus/penyebut), 2))
        return (hasil)
        print(round(hasil, 2))
def getMax(l):
    v=list(l.values())
    k=list(l.keys())
    return k[v.index(max(v))]

In [31]:
pointRow = 'HASIL'

# Entropy(s)
totalPointRow = df['HASIL'].value_counts().tolist()
keyPointRow = df['HASIL'].value_counts().index.tolist()

# hasil dari entropy(s)
plus = totalPointRow[0]
minus = totalPointRow[1]
entropyS = entropy(plus, minus)
print('entropy(s) : ', entropyS)

head = list(df.columns)
head.remove("HASIL")
print(head)

arrGain = {}
for h in head:
    cek = cekDigit(df[h])
    if (cek):
        count = df[h].count()
        sum = df[h].sum()
        median = round(sum/count, 1)
        
        value = df[h].tolist()
        valueHasil = df["HASIL"].tolist()
        
#         print(value)
        
        countBiggerLolos = 0
        countLowerLolos = 0
        countBiggerGagal = 0
        countLowerGagal = 0
        
        for i, val in enumerate(value):
            if (value[i] >= median):
                if (valueHasil[i] == 1):
#                     lolos
                    countBiggerLolos = countBiggerLolos + 1
                else:
#                     tidak lolos
                    countBiggerGagal = countBiggerGagal + 1
            else:
                if (valueHasil[i] == 1):
#                     lolos
                    countLowerLolos = countLowerLolos + 1
                else:
#                     tidak lolos
                    countLowerGagal = countLowerGagal + 1
        
        entropyBigger = entropy(countBiggerLolos, countBiggerGagal)
        entropyLower = entropy(countLowerLolos, countLowerGagal)
        
#         hitung gain
        gain = round(entropyS,2) - round((((countBiggerLolos+countBiggerGagal)/30 * round(entropyBigger, 2)) + ((countLowerLolos+countLowerGagal)/30 * round(entropyLower, 2))), 2)
        arrGain.update({h: gain})
        print(h, " ", round(gain, 2))
    else:
        value = df[h].value_counts().index.tolist()
        arrEntropy = []
        arrPembilang = []
        for v in value:
            hasilLolos = df["HASIL"] == 1  
            hasilGagal = df["HASIL"] == 0
            cond = df[h] == v
            
            dfLolos = df[hasilLolos & cond]
            dfGagal = df[hasilGagal & cond]
            
            plus = dfLolos['HASIL'].count()
            minus = dfGagal['HASIL'].count()
            
            en = entropy(plus, minus)
#             print(v, " : ", en)
            arrEntropy.append(en)
            arrPembilang.append(plus + minus)
        
#         hitung gain
        countRight = 0
        for i, arr in enumerate(arrEntropy):
            countRight = countRight + ( arrPembilang[i]/30 *  arrEntropy[i])
            
        gain = round(entropyS, 2) - countRight
        arrGain.update({h: gain})
        print(h, " ", round(gain, 2))
    
print("list gain : ", arrGain)

entropy(s) :  0.9480782435939054
['DIVISI', 'PENDIDIKAN', 'PRODI', 'IPK', 'PEMBERKASAN', 'WAWANCARA', 'AGE', 'STATUS']
DIVISI   0.07
PENDIDIKAN   0.08
PRODI   0.08
IPK   0.37
PEMBERKASAN   0.06
WAWANCARA   0.22
AGE   0.07
STATUS   0.03
list gain :  {'DIVISI': 0.06785355177632812, 'PENDIDIKAN': 0.07774863336535998, 'PRODI': 0.07821289082932859, 'IPK': 0.37, 'PEMBERKASAN': 0.05892181005228747, 'WAWANCARA': 0.22023135418035356, 'AGE': 0.06999999999999995, 'STATUS': 0.03354124755588561}
